In [13]:
import random
import heapq
import time

# Define as dimensões do tabuleiro
N = 3

# Estrutura para armazenar um estado do puzzle
class EstadoPuzzle:
    def __init__(self, tabuleiro, x, y, profundidade, custo, pai=None):
        self.tabuleiro = tabuleiro
        self.x = x
        self.y = y
        self.profundidade = profundidade  # g(n): Custo acumulado
        self.custo = custo  # f(n) = g(n) + h(n)
        self.pai = pai

    def __lt__(self, outro):
        return self.custo < outro.custo

# Movimentos possíveis: Esquerda, Direita, Cima, Baixo
movimentos = [(0, -1), (0, 1), (-1, 0), (1, 0)]
nomes_movimentos = ["Esquerda", "Direita", "Cima", "Baixo"]

# Função para verificar se um estado é o objetivo
objetivo = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
def estado_objetivo(tabuleiro):
    return tabuleiro == objetivo

# Função para verificar se um movimento é válido
def movimento_valido(x, y):
    return 0 <= x < N and 0 <= y < N

# Heurística: Contagem de peças fora do lugar
def heuristica(tabuleiro):
    return sum(1 for i in range(N) for j in range(N) if tabuleiro[i][j] != 0 and tabuleiro[i][j] != objetivo[i][j])

# Função para contar inversões no tabuleiro
def contar_inversoes(tabuleiro):
    lista_plana = [num for linha in tabuleiro for num in linha if num != 0]
    return sum(1 for i in range(len(lista_plana)) for j in range(i + 1, len(lista_plana)) if lista_plana[i] > lista_plana[j])

# Função para verificar se o tabuleiro é solucionável
def e_solucionavel(tabuleiro):
    return contar_inversoes(tabuleiro) % 2 == 0

# Função para gerar um tabuleiro inicial aleatório e solucionável
def novo_tabuleiro_aleatorio():
    while True:
        numeros = list(range(N * N))
        random.shuffle(numeros)
        matriz = [numeros[i:i+N] for i in range(0, N * N, N)]
        if e_solucionavel(matriz):
            return matriz

# Função para permitir que o usuário insira um tabuleiro inicial
def criar_tabuleiro_definido():
    print("Insira o tabuleiro linha por linha, inserindo os números separados por espaço.")
    print("Exemplo para a primeira linha: 1 2 3")
    tabuleiro = []
    for i in range(N):
        while True:
            try:
                linha = list(map(int, input(f"Inserir linha {i+1}: ").split()))
                if len(linha) != N:
                    raise ValueError
                tabuleiro.append(linha)
                break
            except ValueError:
                print("Entrada inválida. Por favor, insira 3 números separados por espaço.")
    return tabuleiro

# Função para gravar a solução em um arquivo
def escrever_arquivo(tabuleiros, passos, tempo, inicio):
    with open("solucaoAEstrela.txt", "w") as arquivo:
        arquivo.write(f"Total de passos: {passos}\n")
        arquivo.write(f"Tempo de processamento: {tempo:.4f} segundos\n")
        arquivo.write("Tabuleiro inicial:\n")
        
        for linha in inicio:
            arquivo.write(" ".join(map(str, linha)) + "\n")  # Escreve o tabuleiro inicial corretamente
            
        arquivo.write("--------\n")
        arquivo.write("Sequência de movimentos:\n")
       
        for i, tabuleiro in enumerate(tabuleiros):
            arquivo.write(f"Passo {i}:\n")
            for linha in tabuleiro:
                arquivo.write(" ".join(map(str, linha)) + "\n")
            arquivo.write("--------\n")

# Função para obter o caminho da solução
def obter_caminho_solucao(estado):
    caminho = []
    while estado:
        caminho.append(estado.tabuleiro)
        estado = estado.pai
    return caminho[::-1]

# Função principal de busca A*
def resolver_puzzle_a_estrela(inicio, x, y):
    tempo_inicio = time.time()
    fila_prioridade = []
    visitados = set()

    raiz = EstadoPuzzle(inicio, x, y, 0, heuristica(inicio))
    heapq.heappush(fila_prioridade, raiz)
    visitados.add(tuple(map(tuple, inicio)))

    while fila_prioridade:
        atual = heapq.heappop(fila_prioridade)

        if estado_objetivo(atual.tabuleiro):
            tempo_fim = time.time()
            tempo_decorrido = tempo_fim - tempo_inicio
            caminho_solucao = obter_caminho_solucao(atual)
            escrever_arquivo(caminho_solucao, atual.profundidade, tempo_decorrido, inicio)
            return

        for i, (dx, dy) in enumerate(movimentos):
            novo_x, novo_y = atual.x + dx, atual.y + dy
            if movimento_valido(novo_x, novo_y):
                novo_tabuleiro = [linha[:] for linha in atual.tabuleiro]
                novo_tabuleiro[atual.x][atual.y], novo_tabuleiro[novo_x][novo_y] = novo_tabuleiro[novo_x][novo_y], novo_tabuleiro[atual.x][atual.y]
                if tuple(map(tuple, novo_tabuleiro)) not in visitados:
                    visitados.add(tuple(map(tuple, novo_tabuleiro)))
                    filho = EstadoPuzzle(novo_tabuleiro, novo_x, novo_y, atual.profundidade + 1, atual.profundidade + 1 + heuristica(novo_tabuleiro), atual)
                    heapq.heappush(fila_prioridade, filho)



# Execução do programa
if __name__ == "__main__":
    
    escolha = input("Escolha uma opção:\n1 - Inserir tabuleiro manualmente\n2 - Gerar tabuleiro aleatório\nOpção: ")

    if escolha == "1":
        tabuleiro_inicial = criar_tabuleiro_definido()
    else:
        tabuleiro_inicial = novo_tabuleiro_aleatorio()

    x, y = next((i, j) for i in range(N) for j in range(N) if tabuleiro_inicial[i][j] == 0)

    print("Estado Inicial:")
    for linha in tabuleiro_inicial:
        print(" ".join(map(str, linha)))
    print("--------")

    resolver_puzzle_a_estrela(tabuleiro_inicial, x, y)


Estado Inicial:
5 3 1
7 6 4
2 8 0
--------
